In [ ]:
import srsly
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
public = pd.read_csv("/workspace/resources/data/dump-20230630/set_de_datos_con_perspectiva_de_genero.csv")

public.dropna(subset=["NRO_REGISTRO", "TOMO", "FECHA_RESOLUCION"], inplace=True)
public.rename(columns={c: c.lower() for c in public.columns}, inplace=True)

public["tomo"] = public["tomo"].astype(int)


def fix_date(text: str):
    date = text.split("_")
    # print(date)
    if len(date) != 3:
        return

    for i, num in enumerate(date):
        num = int(num)
        date[i] = f"{num:02.0f}"
    try:
        return pd.to_datetime("/".join(date))
    except:
        return


public["fecha_resolucion"] = [fix_date(date) for date in tqdm(public["fecha_resolucion"])]

public

In [ ]:
public.info()

In [ ]:
from aymurai.datasets.ar_juz_pcyf_10.annotations import (
    ArgentinaJuzgadoPCyF10LabelStudioAnnotations,
)

annotations = ArgentinaJuzgadoPCyF10LabelStudioAnnotations("/workspace/resources/annotations")

In [ ]:
from aymurai.utils.misc import get_element


def text_normalize(text: str):
    text = text.replace("\/", "/")
    return text


metadata = pd.DataFrame(
    [
        x["metadata"]
        | x["data"]
        | {
            "original_path": x["path"],
            "conll": get_element(x, levels=["annotations", "conll"]),
        }
        for x in annotations.data
    ]
)
metadata.dropna(subset=["tomo"], inplace=True)
metadata = metadata[metadata["tomo"].str.isnumeric()]
metadata["tomo"] = metadata["tomo"].astype(int)
metadata["original_path"] = metadata["original_path"].apply(text_normalize)
metadata["doc.text"] = metadata["doc.text"].apply(text_normalize)

# metadata['tomo'] = metadata['tomo'].astype(int)
metadata.drop_duplicates(["tomo", "nro_registro"], inplace=True)
metadata

In [ ]:
# Left join
x = pd.merge(public, metadata, on=["nro_registro", "tomo"], how="left")
x.drop_duplicates(["tomo", "nro_registro"], inplace=True)
x.info()

In [ ]:
x.sample(5)

In [ ]:
x["link"].notna().sum()

In [ ]:
x.groupby(x["fecha_resolucion"].dt.to_period("Y"))["link"].count()

In [ ]:
x.dropna(subset=["original_path"]).groupby(x["fecha_resolucion"].dt.to_period("Y"))["original_path"].count()

In [ ]:
import os
import gdown
import subprocess
from hashlib import blake2b
from tqdm.auto import tqdm
from joblib import Parallel, delayed

tqdm.pandas()

outdir = "/resources/data/documents"
os.makedirs(outdir, exist_ok=True)


def get_file(url: str):
    if not isinstance(url, str):
        return

    fname = f"{outdir}/{blake2b(url.encode(), digest_size=15).hexdigest()}"

    if os.path.exists(fname):
        return fname

    # # algunos archivos requieren acceso, otros estan corruptos
    # cmd = f"gdown --fuzzy -q --continue -O {fname} {url}"
    try:
        pass
        # fname = gdown.download(url, fname, quiet=False, fuanon_tfidfzzy=True, resume=True)
        # output = subprocess.getoutput(cmd)
    except:
        return

    # if "Access denied" in output:
    # return

    return fname


parallel = Parallel(n_jobs=50, backend="threading")
get_file_ = delayed(get_file)
path = parallel(get_file_(url) for url in tqdm(x["link"]))

In [ ]:
x["anonymized_path"] = path

In [ ]:
x.sample(5)

In [ ]:
x["anonymized_path"].notna().sum(), x["anonymized_path"].isna().sum()

In [ ]:
x.dropna(subset=["anonymized_path"]).groupby(x["fecha_resolucion"].dt.to_period("Y"))["anonymized_path"].count()

In [ ]:
x["original_path"].notna().sum(), x["original_path"].isna().sum()

In [ ]:
x.dropna(subset=["original_path"]).groupby(x["fecha_resolucion"].dt.to_period("Y"))["original_path"].count()

In [ ]:
# 2021 - 2022, con versión anonimizada y sin versión original
mask = (
    (x["fecha_resolucion"].dt.to_period("Y") >= "2021")
    & (x["fecha_resolucion"].dt.to_period("Y") <= "2022")
    & (x["anonymized_path"].notna())
    & (x["original_path"].isna())
)
x = x.loc[mask]
x

In [ ]:
x.groupby(x["fecha_resolucion"].dt.to_period("Y"))["anonymized_path"].count()

In [ ]:
import textract

IA2_START_DATE = pd.to_datetime("01/01/2021")


def load_doc(path: str):
    doc = textract.process(path, extension="odt")
    return doc.decode("utf-8")


def ia2(row):
    path = row["anonymized_path"]
    date = row["fecha_resolucion"]
    if date < IA2_START_DATE:
        return False
    try:
        doc = load_doc(path)
    except Exception as e:
        return False
    return "XX" not in doc

In [ ]:
x["ia2"] = [ia2(row) for i, row in tqdm(x.iterrows())]

In [ ]:
x.query("ia2")["fecha_resolucion"].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])

In [ ]:
# Filtro de resoluciones sin versión original pero con versión anonymizada
mask &= (x["original_path"].isna()) & (x["ia2"])
x = x.loc[mask]
x.info()

In [ ]:
x.loc[mask]["fecha_resolucion"].describe()

In [ ]:
x["anonymized_path"].count()

In [ ]:
x["anonymized_path"].isna().sum()

In [ ]:
x["anonymized_text"] = x["anonymized_path"].map(load_doc)

In [ ]:
x[["anonymized_path", "anonymized_text"]].sample()

In [ ]:
x["anonymized_text"].info()

In [ ]:
from aymurai.text.normalize import document_normalize

In [ ]:
x["anonymized_text"] = x["anonymized_text"].map(text_normalize).map(document_normalize)

In [ ]:
from glob import glob

originals = glob(
    "/resources/data/restricted/ar-juz-pcyf-10/RESOLUCIONES DEL JUZGADO/**/**.doc",
    recursive=True,
) + glob(
    "/resources/data/restricted/ar-juz-pcyf-10/RESOLUCIONES DEL JUZGADO/**/**.docx",
    recursive=True,
)

len(originals)

In [ ]:
originals[-5:]

In [ ]:
original_paths = (
    metadata["original_path"]
    .str.replace("/resources/", "/resources/data/")
    .str.replace("/RESOLUCIONES DEL JUZGADO - DOCS/", "/RESOLUCIONES DEL JUZGADO/")
    .unique()
)

In [ ]:
originals = [original for original in originals if original not in original_paths]
len(originals)

In [ ]:
def load_word_doc(path: str):
    try:
        doc = (
            textract.process(path, extension="doc")
            if path.endswith(".doc")
            else textract.process(path, extension="docx")
        )

    except:
        return

    return doc.decode("utf-8")

In [ ]:
original_docs = list(map(load_word_doc, originals))

In [ ]:
for i, doc in enumerate(original_docs):
    if not doc:
        originals.pop(i)
        original_docs.pop(i)

In [ ]:
none_idx = original_docs.index(None)

originals.pop(none_idx)
original_docs.pop(none_idx)

In [ ]:
original_docs = list(map(text_normalize, original_docs))
original_docs = list(map(document_normalize, original_docs))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer()
anon_tfidf = tfidf.fit_transform(x["anonymized_text"])
anon_tfidf

In [ ]:
orig_tfidf = tfidf.transform(original_docs)
orig_tfidf

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarities = cosine_similarity(anon_tfidf, orig_tfidf)
cosine_similarities.shape

In [ ]:
import numpy as np

In [ ]:
np.argmax(cosine_similarities, axis=1)

In [ ]:
print(x["anonymized_text"].iloc[0])

In [ ]:
print(original_docs[2028])

In [ ]:
matching = x[
    [
        "nro_registro",
        "tomo",
        "fecha_resolucion",
        "n_expte_eje",
        "link",
        "doc.text",
        "original_path",
        "anonymized_path",
        "anonymized_text",
    ]
]
matching

In [ ]:
matching["max_similarity"] = np.max(cosine_similarities, axis=1)
matching

In [ ]:
matching["max_similarity"].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])

In [ ]:
matching["max_similarity"].plot(kind="hist", bins=30)

In [ ]:
matching["argmax_similarity"] = np.argmax(cosine_similarities, axis=1)
matching

In [ ]:
matching["matched_text"] = matching["argmax_similarity"].map(lambda x: original_docs[x])
matching

In [ ]:
matching["matched_path"] = matching["argmax_similarity"].map(lambda x: originals[x])
matching

In [ ]:
matching.info()

In [ ]:
q1 = matching["max_similarity"].quantile(0.01)
q5 = matching["max_similarity"].quantile(0.05)
q25 = matching["max_similarity"].quantile(0.25)
q50 = matching["max_similarity"].quantile(0.5)

for i, r in matching.query("max_similarity > @q25 and max_similarity <= @q50").iterrows():
    print(r["max_similarity"])
    print(r["matched_text"])
    print("=" * 225)
    print(r["anonymized_text"])
    print("=" * 225)
    print("*" * 225)

```
# Distribución de similitud de los documentos que tenemos emparejados
count    252.000000
mean       0.955130
std        0.045909
min        0.629422
1%         0.794987
5%         0.883193
25%        0.939891
50%        0.969540
75%        0.984278
95%        0.993554
99%        0.996443
max        0.996680
Name: max_similarity, dtype: float64
```

In [ ]:
# Tomamos una similitud mayor al percentil 1 de la distriución conocida
matching.query("max_similarity > 0.795").describe()

In [ ]:
from scipy import stats

percentile = stats.percentileofscore(matching["max_similarity"], 0.795)
percentile

In [ ]:
for i, r in matching.query("max_similarity > 0.795").iterrows():
    print(r["max_similarity"])
    print(r["matched_text"])
    print("=" * 225)
    print(r["anonymized_text"])
    print("=" * 225)
    print("*" * 225)

In [ ]:
matching[["matched_text", "anonymized_path"]].rename(columns={"matched_text": "doc.text"})

In [ ]:
from aymurai.utils import alignment

# OUTPUT_DIR = "/resources/data/restricted/anonymization/ia2-aligned"
# os.makedirs(OUTPUT_DIR, exist_ok=True)

data = matching.drop(columns=["doc.text"]).rename(columns={"matched_text": "doc.text"}).copy()
mappings = {}

for i, row in tqdm(data.iterrows(), total=len(data)):
    mapping = alignment.process(row)
    mappings[i] = mapping
    data.loc[i, "percentage_matched"] = (mapping["original"] == mapping["anonymized"]).mean()
    # filename = f"{OUTPUT_DIR}/{row['tomo']}_{row['nro_registro']}.csv"
    # mapping.to_csv(filename)

In [ ]:
data.sample(5)

In [ ]:
data["percentage_matched"].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])

In [ ]:
data["percentage_matched"].plot(kind="hist", bins=30)

In [ ]:
data.sort_values("percentage_matched", ascending=False, inplace=True)

In [ ]:
data.head(10)

In [ ]:
data.tail(10)

In [ ]:
data["score"] = data["max_similarity"] * data["percentage_matched"]
data["score"].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])

In [ ]:
data["score"].plot(kind="hist", bins=30)

In [ ]:
data_filtered = data.query("score >= 0.68")
data_filtered.info()

In [ ]:
data_filtered["percentage_matched"].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])

In [ ]:
np.random.choice(data_filtered.index)

In [ ]:
from IPython.display import HTML

rnd_idx = np.random.choice(data_filtered.index)

print(data_filtered.loc[rnd_idx, "doc.text"])

HTML(mappings[rnd_idx].to_html())